# Lab 9.1: MultiQueryDemo

## Tổng quan bài tập
**Đề bài**: Ở bài Lab này, bạn sẽ được hướng dẫn các thao tác về việc Đọc dữ liệu Stream từ Kafka cũng như cách ghi dữ liệu ở cả dạng File Sink và Kafka Sink

## Tài nguyên
Do bài Lab này liên quan đến đến xử lý dữ liệu Stream với Kafka, vậy nên bạn sẽ cần cài đặt Kafka, bạn có thể tham khảo video sau về cách cài đặt:
- [Cài đặt Kafka](https://funix.udemy.com/course/spark-streaming-using-python/learn/lecture/21955580#overview)

Bạn sẽ cần tải các Kafka Script ở [link sau](https://drive.google.com/file/d/1RMiLzXVTiRlNsvfF3I63ylKVGjBXt7AK/view?usp=sharing) để có thể sử dụng Kafka.


Bạn cũng sẽ cần tải các dữ liệu ở [link sau](https://drive.google.com/file/d/18RrmmF1h2-HS6wvteZ5lPcxs4iPCm5CB/view?usp=sharing) để có thể kiểm thử cho bài Lab. 


Ngoài ra, bạn có thể tham khảo các video sau trong trường hợp chưa hiểu cách làm bài Lab:
- [Multi-query Streams Application](https://funix.udemy.com/course/spark-streaming-using-python/learn/lecture/21955620#overview)

Import các Package cần thiết

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import from_json, col, expr
from pyspark.sql.types import StructType, StructField, StringType, LongType, DoubleType, IntegerType, ArrayType

In [2]:
spark = SparkSession \
	.builder \
	.appName("lab9 Multi Query Demo") \
	.master("local[3]") \
	.config("spark.streaming.stopGracefullyOnShutdown", "true") \
	.getOrCreate()

24/03/09 20:50:15 WARN Utils: Your hostname, VNHCM1QANB017 resolves to a loopback address: 127.0.1.1; using 172.31.49.251 instead (on interface eth0)
24/03/09 20:50:15 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/home/baodk/Programs/spark-3.5.1-bin-hadoop3-scala2.13/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/baodk/.ivy2/cache
The jars for the packages stored in: /home/baodk/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.13 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-1607c9ff-7f31-484f-8453-4f2276642cf1;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.13;3.5.1 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.13;3.5.1 in central
	found org.apache.kafka#kafka-clients;3.4.1 in central
	found org.lz4#lz4-java;1.8.0 in central
	found org.xerial.snappy#snappy-java;1.1.10.3 in central
	found org.slf4j#slf4j-api;2.0.7 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.4 in central
	found org.apache.hadoop#hadoop-client-api;3.3.4 in central
	found commons-logging#commons-logging;1.1.3 in central
	found com.google.code.findbugs#jsr305;3.0.0 in central
	found org.scala-lang.modules#scala-parallel-collections_2.13;1.0.4 in central
	found org.apache.commons#commons-pool2;2.1

In [ ]:
spark.stop()

In [3]:
print(spark._jsc.sc().getConf().get("spark.jars"))

file:///home/baodk/.ivy2/jars/org.apache.spark_spark-sql-kafka-0-10_2.13-3.5.1.jar,file:///home/baodk/.ivy2/jars/org.apache.spark_spark-token-provider-kafka-0-10_2.13-3.5.1.jar,file:///home/baodk/.ivy2/jars/org.scala-lang.modules_scala-parallel-collections_2.13-1.0.4.jar,file:///home/baodk/.ivy2/jars/org.apache.kafka_kafka-clients-3.4.1.jar,file:///home/baodk/.ivy2/jars/com.google.code.findbugs_jsr305-3.0.0.jar,file:///home/baodk/.ivy2/jars/org.apache.commons_commons-pool2-2.11.1.jar,file:///home/baodk/.ivy2/jars/org.apache.hadoop_hadoop-client-runtime-3.3.4.jar,file:///home/baodk/.ivy2/jars/org.lz4_lz4-java-1.8.0.jar,file:///home/baodk/.ivy2/jars/org.xerial.snappy_snappy-java-1.1.10.3.jar,file:///home/baodk/.ivy2/jars/org.slf4j_slf4j-api-2.0.7.jar,file:///home/baodk/.ivy2/jars/org.apache.hadoop_hadoop-client-api-3.3.4.jar,file:///home/baodk/.ivy2/jars/commons-logging_commons-logging-1.1.3.jar


Tạo schema cho dữ liệu đầu vào

In [4]:
schema = StructType([
	StructField("InvoiceNumber", StringType()),
	StructField("CreatedTime", LongType()),
	StructField("StoreID", StringType()),
	StructField("PosID", StringType()),
	StructField("CashierID", StringType()),
	StructField("CustomerType", StringType()),
	StructField("CustomerCardNo", StringType()),
	StructField("TotalAmount", DoubleType()),
	StructField("NumberOfItems", IntegerType()),
	StructField("PaymentMethod", StringType()),
	StructField("CGST", DoubleType()),
	StructField("SGST", DoubleType()),
	StructField("CESS", DoubleType()),
	StructField("DeliveryType", StringType()),
	StructField("DeliveryAddress", StructType([
		StructField("AddressLine", StringType()),
		StructField("City", StringType()),
		StructField("State", StringType()),
		StructField("PinCode", StringType()),
		StructField("ContactNumber", StringType())
	])),
	StructField("InvoiceLineItems", ArrayType(StructType([
		StructField("ItemCode", StringType()),
		StructField("ItemDescription", StringType()),
		StructField("ItemPrice", DoubleType()),
		StructField("ItemQty", IntegerType()),
		StructField("TotalValue", DoubleType())
	]))),
])

Hãy hoàn thiện các phần `[...]` để hoàn thiện đoạn code và giải quyết bài toán theo yêu cầu.

In [ ]:
# Đọc dữ liệu từ Kafka
kafka_df = spark.readStream \
	.format("kafka") \
	.option("kafka.bootstrap.servers", "localhost:9092") \
	.option("subscribe", "invoices") \
	.option("startingOffsets", "earliest") \
	.load()

# Chuyển dữ liệu từ dạng JSON về MapType()
value_df = kafka_df.select(from_json(col("value").cast("string"), schema).alias("value"))

# Xử lý dữ liệu
notification_df = value_df.select("value.InvoiceNumber", "value.CustomerCardNo", "value.TotalAmount") \
	.withColumn("EarnedLoyaltyPoints", expr("TotalAmount * 0.2"))

# Ghi dữ liệu ở dạng Kafka Sink
# Chuyển đổi Dataframe về dạng key - value
kafka_target_df = notification_df.selectExpr("InvoiceNumber as key", "to_json(struct(*)) as value")

notification_writer_query = kafka_target_df \
	.writeStream \
	.queryName("Notification Writer") \
	.format("kafka") \
	.option("kafka.bootstrap.servers", "localhost:9092") \
	.option("topic", "notifications") \
	.outputMode("append") \
	.option("checkpointLocation", "chk-point-dir/notify") \
	.start()

# Trích xuất các dữ liệu
explode_df = value_df.selectExpr("value.InvoiceNumber", "value.CreatedTime", "value.StoreID",
									"value.PosID", "value.CustomerType", "value.PaymentMethod", "value.DeliveryType",
									"value.DeliveryAddress.City",
									"value.DeliveryAddress.State", "value.DeliveryAddress.PinCode",
									"explode(value.InvoiceLineItems) as LineItem")

flattened_df = explode_df \
	.withColumn("ItemCode", expr("LineItem.ItemCode")) \
	.withColumn("ItemDescription", expr("LineItem.ItemDescription")) \
	.withColumn("ItemPrice", expr("LineItem.ItemPrice")) \
	.withColumn("ItemQty", expr("LineItem.ItemQty")) \
	.withColumn("TotalValue", expr("LineItem.TotalValue")) \
	.drop("LineItem")


# Ghi dữ liệu dưới dạng File Sink
invoice_writer_query = flattened_df.writeStream \
	.format("json") \
	.queryName("Flattened Invoice Writer") \
	.outputMode("append") \
	.option("path", "output") \
	.option("checkpointLocation", "chk-point-dir/flatten") \
	.start()

print("Waiting for Queries")
spark.streams.awaitAnyTermination()

24/03/09 20:54:00 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
24/03/09 20:54:01 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


Waiting for Queries


24/03/09 20:54:01 WARN AdminClientConfig: These configurations '[key.deserializer, value.deserializer, enable.auto.commit, max.poll.records, auto.offset.reset]' were supplied but are not used yet.
24/03/09 20:54:01 WARN AdminClientConfig: These configurations '[key.deserializer, value.deserializer, enable.auto.commit, max.poll.records, auto.offset.reset]' were supplied but are not used yet.
